In [ ]:
import tensorflow as tf 
import os 

import tensorflow_model_analysis as tfma
from tfx.components import (
    CsvExampleGen, 
    StatisticsGen, 
    SchemaGen, 
    ExampleValidator, 
    Transform, 
    Trainer, 
    Tuner,
    Evaluator, 
    Pusher
    )
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import (
    LatestBlessedModelStrategy
    )


SCHEMA_PIPELINE_NAME = 'sleep-stress-tfdv-schema'
PIPELINE_NAME = 'sleep-stress-pipeline'

SCHEMA_PIPELINE_ROOT = os.path.join('pipelines', SCHEMA_PIPELINE_NAME)
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

SCHEMA_METADATA_PATH = os.path.join('metadata', SCHEMA_PIPELINE_NAME, 'metadata.db')
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

DATA_ROOT = '/home/gfspet/mlops-dicoding/sub2/dataset'

interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [118]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2)
        ])
    )
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [119]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
  )

interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [120]:
interactive_context.show(statistics_gen.outputs['statistics'])

In [121]:
schema_gen = SchemaGen(
      statistics=statistics_gen.outputs['statistics'], 
      infer_feature_shape=True
    )

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [122]:
interactive_context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'blood_oxygen',FLOAT,required,,-
'body_temp',FLOAT,required,,-
'eye_movement',FLOAT,required,,-
'heart_rate',FLOAT,required,,-
'limb_movement',FLOAT,required,,-
'respiration_rate',FLOAT,required,,-
'sleep_hours',FLOAT,required,,-
'snoring_rate',FLOAT,required,,-
'stress_rate',STRING,required,,'stress_rate'


,Values
Domain,
'stress_rate',"'high/unhealthy', 'low/normal', 'medium', 'medium_high', 'medium_low'"


In [123]:
example_validator = ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema']
    )
  
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [124]:
interactive_context.show(example_validator.outputs['anomalies'])

In [ ]:
transform  = Transform(
        examples=example_gen.outputs['examples'],
        schema= schema_gen.outputs['schema'],
        module_file=os.path.abspath('stress_sleep_transform.py')
    )

interactive_context.run(transform)

running bdist_wheel
running build
running build_py
creating build/lib
copying stress_sleep_transform.py -> build/lib
copying stress_sleep_tuner.py -> build/lib
copying stress_sleep_trainer.py -> build/lib
copying main_pipeline.py -> build/lib
installing to /tmp/tmpi_w4esy2
running install
running install_lib
copying build/lib/stress_sleep_transform.py -> /tmp/tmpi_w4esy2/.
copying build/lib/stress_sleep_tuner.py -> /tmp/tmpi_w4esy2/.
copying build/lib/stress_sleep_trainer.py -> /tmp/tmpi_w4esy2/.
copying build/lib/main_pipeline.py -> /tmp/tmpi_w4esy2/.
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manif

/home/gfspet/miniconda3/envs/tfx-tfdf/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipelines/sleep-stress-pipeline/_wheels/tfx_user_code_Transform-0.0+2ea9395967d50f42e760dab2534565e3c889c88ae69387126828e863700f894b-py3-none-any.whl
'high/unhealthy' is encoded as 0
'low/normal' is encoded as 1
'medium' is encoded as 2
'medium_high' is encoded as 3
'medium_low' is encoded as 4
Processing ./pipelines/sleep-stress-pipeline/_wheels/tfx_user_code_Transform-0.0+2ea9395967d50f42e760dab2534565e3c889c88ae69387126828e863700f894b-py3-none-any.whl
'high/unhealthy' is encoded as 0
'low/normal' is encoded as 1
'medium' is encoded as 2
'medium_high' is encoded as 3
'medium_low' is encoded as 4
Processing ./pipelines/sleep-stress-pipeline/_wheels/tfx_user_code_Transform-0.0+2ea9395967d50f42e760dab2534565e3c889c88ae69387126828e863700f894b-py3-none-any.whl


INFO:tensorflow:Assets written to: pipelines/sleep-stress-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/2c5201f656c447caa3c0d0dfe98b99a0/assets


INFO:tensorflow:Assets written to: pipelines/sleep-stress-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/2c5201f656c447caa3c0d0dfe98b99a0/assets


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [126]:
transform.outputs

{'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, producer_componen

In [127]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transform_fn', 'metadata', 'transformed_metadata']

In [128]:
import pprint

# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pprint.PrettyPrinter().pprint(example)

features {
  feature {
    key: "blood_oxygen_xf"
    value {
      float_list {
        value: -0.2707175016403198
      }
    }
  }
  feature {
    key: "body_temp_xf"
    value {
      float_list {
        value: -0.27112695574760437
      }
    }
  }
  feature {
    key: "eye_movement_xf"
    value {
      float_list {
        value: 0.9318127632141113
      }
    }
  }
  feature {
    key: "heart_rate_xf"
    value {
      float_list {
        value: 0.9697924256324768
      }
    }
  }
  feature {
    key: "limb_movement_xf"
    value {
      float_list {
        value: 1.133789300918579
      }
    }
  }
  feature {
    key: "respiration_rate_xf"
    value {
      float_list {
        value: 0.9697930216789246
      }
    }
  }
  feature {
    key: "sleep_hours_xf"
    value {
      float_list {
        value: -0.6042509078979492
      }
    }
  }
  feature {
    key: "snoring_rate_xf"
    value {
      float_list {
        value: 1.1388148069381714
      }
    }
  }
  feature {

'high/unhealthy' is encoded as 0
'low/normal' is encoded as 1
'medium' is encoded as 2
'medium_high' is encoded as 3
'medium_low' is encoded as 4

In [ ]:
tuner = Tuner(
    module_file=os.path.abspath('stress_sleep_tuner.py'),  
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(
        splits=['train'], 
        num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(
        splits=['eval'],
        num_steps=500
    )
)

interactive_context.run(tuner)

Trial 20 Complete [00h 00m 02s]
val_accuracy: 0.5245901346206665

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 00m 38s
Results summary
Results in pipelines/sleep-stress-pipeline/.temp/6/kt_tuner
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 01 summary
Hyperparameters:
units: 384
drop_rate: 0.1
lr: 0.001
Score: 1.0

Trial 04 summary
Hyperparameters:
units: 160
drop_rate: 0.1
lr: 0.001
Score: 1.0

Trial 06 summary
Hyperparameters:
units: 352
drop_rate: 0.1
lr: 0.001
Score: 1.0

Trial 07 summary
Hyperparameters:
units: 288
drop_rate: 0.30000000000000004
lr: 0.001
Score: 1.0

Trial 12 summary
Hyperparameters:
units: 448
drop_rate: 0.1
lr: 0.001
Score: 1.0

Trial 15 summary
Hyperparameters:
units: 192
drop_rate: 0.1
lr: 0.001
Score: 1.0

Trial 17 summary
Hyperparameters:
units: 512
drop_rate: 0.4
lr: 0.001
Score: 1.0

Trial 18 summary
Hyperparameters:
units: 288
drop_rate: 0.4
lr: 0.001
Score: 1.0

Trial 02 summary
Hyperparameters:
units: 320
drop_ra

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [ ]:
trainer = Trainer(
    module_file=os.path.abspath('stress_sleep_trainer.py'),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(
        splits=['train'],
        num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(
        splits=['eval'],
        num_steps=500)
)

interactive_context.run(trainer)

running bdist_wheel
running build
running build_py
creating build/lib
copying stress_sleep_transform.py -> build/lib
copying stress_sleep_tuner.py -> build/lib
copying stress_sleep_trainer.py -> build/lib
copying main_pipeline.py -> build/lib
installing to /tmp/tmpo780nrzi
running install
running install_lib
copying build/lib/stress_sleep_transform.py -> /tmp/tmpo780nrzi/.
copying build/lib/stress_sleep_tuner.py -> /tmp/tmpo780nrzi/.
copying build/lib/stress_sleep_trainer.py -> /tmp/tmpo780nrzi/.
copying build/lib/main_pipeline.py -> /tmp/tmpo780nrzi/.
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tf

/home/gfspet/miniconda3/envs/tfx-tfdf/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipelines/sleep-stress-pipeline/_wheels/tfx_user_code_Trainer-0.0+2ea9395967d50f42e760dab2534565e3c889c88ae69387126828e863700f894b-py3-none-any.whl
{'units': 384, 'drop_rate': 0.1, 'lr': 0.001}
Epoch 1/10
 153/1000 [===>..........................] - ETA: 4s - loss: 0.2040 - accuracy: 0.9389WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 10000 batches). You may need to use the repeat() function when building your dataset.



Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to pipelines/sleep-stress-pipeline/Trainer/model/7/Format-Serving/best_model.h5
1000/1000 [==============================] - 2s 1ms/step - loss: 0.1968 - accuracy: 0.9411 - val_loss: 0.0026 - val_accuracy: 1.0000


/home/gfspet/miniconda3/envs/tfx-tfdf/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines/sleep-stress-pipeline/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/sleep-stress-pipeline/Trainer/model/7/Format-Serving/assets


Model training complete and saved to: pipelines/sleep-stress-pipeline/Trainer/model/7/Format-Serving


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [131]:
model_resolver = Resolver(
    strategy_class = LatestBlessedModelStrategy,
    model = Channel(type = Model),
    model_blessing = Channel(type = ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [ ]:
slicing_specs = [
    tfma.SlicingSpec(), 
    tfma.SlicingSpec(feature_keys=['stress_rate_xf']) 
]


metrics_specs = [
    tfma.MetricsSpec(per_slice_thresholds={
        'accuracy':  
            tfma.PerSliceMetricThresholds(thresholds=[
                tfma.PerSliceMetricThreshold(
                    slicing_specs=[tfma.SlicingSpec()],  
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.6}  
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': -1e-10}
                        )
                    )
                )
            ])
    }),

    tfma.MetricsSpec(metrics=[
        tfma.MetricConfig(class_name='AUC'),
        tfma.MetricConfig(class_name='Precision'),
        tfma.MetricConfig(class_name='Recall'),
        tfma.MetricConfig(class_name='ExampleCount'),
    ])
]

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='stress_rate_xf') 
    ],
    slicing_specs=slicing_specs,  
    metrics_specs=metrics_specs  
)

evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],  
    eval_config=eval_config
)

interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [133]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'accuracy': …

In [134]:
pusher = Pusher(
    model = trainer.outputs['model'],
    model_blessing = evaluator.outputs['blessing'],
    push_destination = pusher_pb2.PushDestination(
        filesystem = pusher_pb2.PushDestination.Filesystem(
            base_directory = 'serving_model_dir/stress_sleep_model'
        )
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))